trash

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../../src/'))
from utils.daps_explorer import DapsExplorer
from model.CNN import DataLoader, Net, SpectrogramDataset
import matplotlib.pyplot as plt
import wandb

from sklearn.metrics import f1_score
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

wandb.login()

root = DapsExplorer("../../../data/daps/")['clean']['script1']

# Use wandb to log training progress

In [ ]:
INPUT_RESOLUTION = 32
LEARNING_RATE = 0.03
LR_DECAY = 0.95
BATCH_SIZE = 16
DATASET_PATH = os.path.join('../../../', 'datasets_big', 'dataset')
print(DATASET_PATH)

In [ ]:
import wandb
wandb.login()
wandb.init(
    project="iml_test",
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_decay": LR_DECAY,
        "batch_size": BATCH_SIZE,
        "input_resolution": INPUT_RESOLUTION,
        # "num_parameters": num_learnable_parameters,
        "architecture": "CNN",
        "dataset": "DAPS"
    }
)


In [ ]:
def train():
    wandb.init(
    project="iml_test",
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_decay": LR_DECAY,
        "batch_size": BATCH_SIZE,
        "input_resolution": INPUT_RESOLUTION,
        # "num_parameters": num_learnable_parameters,
        "architecture": "CNN",
        "dataset": "DAPS"
    }
)
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    epochs_without_val_acc_improvement = 0
    best_val_acc = 0.0

    # Hyperparameters
    batch_size = 64
    learning_rate = 0.001
    num_epochs = 5

    # Prepare Dataset and DataLoader
    train_dataset = SpectrogramDataset(data_dir=os.path.join(DATASET_PATH, "train"),
                                       transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = SpectrogramDataset(data_dir=os.path.join(DATASET_PATH, "validation"),
                                       transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    PRINT_STEP = len(train_loader) // 5 - 1

    model = Net()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        all_labels = []
        all_preds = []

        for  i, data in enumerate(train_loader):
            inputs, labels = data
            # Reshape to match output size [batch_size, 1]
            labels = labels.unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track loss and predictions for F1 score
            preds = (outputs > 0.5).float()  # Binary prediction with threshold 0.5
            all_labels.extend(labels.numpy().flatten())
            all_preds.extend(preds.detach().numpy().flatten())

            correct += (preds == labels).float().mean().item()
            
            running_loss += loss.item()
            if i % PRINT_STEP == PRINT_STEP-1:
                accuracy = correct / PRINT_STEP
                loss = running_loss / PRINT_STEP
                step = epoch * len(train_loader) + i
                wandb.log({
                        "train/accuracy": accuracy,
                        "train/loss": loss
                    },
                    step=step
                )
                running_loss = 0.0
                correct = 0

        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, F1 Score: {f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for j, data in enumerate(val_loader):
                inputs, labels = data
                labels = labels.unsqueeze(1)  # Ensure labels have shape (batch_size, 1)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Binary predictions
                preds = (outputs > 0.5).float()
                val_correct += (preds == labels).float().mean().item()

        accuracy = val_correct / len(val_loader)
        loss = val_loss / len(val_loader)
        wandb.log({
                "validation/accuracy": accuracy,
                "validation/loss": loss
            },
            step=(epoch + 1) * len(train_loader)
        )


        if accuracy > best_val_acc:
            best_val_acc = accuracy
            epochs_without_val_acc_improvement = 0
        else:
            epochs_without_val_acc_improvement += 1
        if epochs_without_val_acc_improvement >= 10:
            print("10 epochs without a val accuracy improvement. Stopping the train")
            break

    torch.save(model, "simple_cnn_full_model.pth")
    wandb.finish()
    print("Training complete.")

train()

In [ ]:
def test():
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Hyperparameters
    batch_size = 64

    test_dataset = SpectrogramDataset(data_dir=os.path.join(DATASET_PATH, "test"),
                                      transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    model = torch.load("simple_cnn_full_model.pth", weights_only=False)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on test images: {100 * correct // total} %')
    
test()